In [4]:
import numpy as np 
import random as rd
import matplotlib.pyplot as plt
import numpy.linalg 
from scipy.optimize import root
from scipy.linalg import svd, sqrtm
from sklearn import linear_model
import scipy.stats as sts



In [5]:
def NIPALS_wikistats(X,Y,K, eps = 1e-9, max_iter = 100):
    n,p = np.shape(X)
    U = []
    V = []
    if centrer_reduire :
        meanx = np.mean(X, axis=0)
        X = X - mean
        meany = np.mean(Y, axis=0)
        Y = Y - mean
    w = Y[:,0].reshape(-1,1)
    for h in range(k):
        diff = 1
        it = 0
        while diff > eps and it < max_iter:
            u = np.dot(X.T,w)/np.linalg.norm(w)
            u = u/np.linalg.norm(u)
            e = X.dot(u)
            v_vieux = v.copy()
            v = Y.T.dot(e)/np.linalg.norm(e)
            om = Y.T.dot(v)
            diff = np.linalg.norm(v-vh)
            it += 1
        U.append(u)
        V.append(V)
        c = X.T.dot(e)/np.linalg.norm(e)
        d = X.T.dot(om)/np.linalg.norm(om)
        X = X - e.dot(c.T)
        Y = Y - om.dot(d.T)
    return U,V

In [12]:
def NIPALS(X,comp, eps = 1e-9,centrer_reduire=False,max_iter = 100):
    n,p = np.shape(X)
    if centrer_reduire :
        mean = np.mean(X, axis=0)
        X = X-mean
        var = np.sqrt(sum((X**2))/n)
        X=X/var
    a = comp
    T = np.zeros((n,a)) #scores
    P = np.zeros((p,a)) #loadings
    lam = []
    Xh = X.copy()
    
    for h in range(a):
        col = np.argmax(np.apply_along_axis(lambda Xh: np.sum(np.abs(Xh), axis=0, keepdims=True), 0,Xh))
        
        th = Xh[:,col].reshape(-1,1)
        
        diff = 1
        p0 = Xh.T.dot(th)/th.T.dot(th).reshape(-1,1)
        k=0
        while diff > eps and k< max_iter :
            ph = Xh.T.dot(th)/th.T.dot(th).reshape(-1,1)
            ph = ph/np.linalg.norm(ph)
            t0 = th.copy()
            th = Xh.dot(ph)/ph.T.dot(ph)
            diff = np.linalg.norm(th-t0)
            k+=1
        Xh = Xh- th.dot(ph.T)
        lh = th.T.dot(th)
        T[:,h]=th[:,0]
        P[:,h]=ph[:,0]
        lam.append(np.sqrt(lh))
        h+=1
    return T,P,lam
def NIPALS_bis(T,P,Y):
    
    c = np.linalg.inv(T.T.dot(T)).dot(T.T).dot(Y)
    return P.dot(c)
def NIPALS_complet(X,Y,k, eps = 1e-9,centrer_reduire=True,max_iter = 100):
    T,P,L = NIPALS(X,k, eps = 1e-9,centrer_reduire=True,max_iter = 100)
    beta = NIPALS_bis(T,P,Y)
    return beta

In [13]:
B=np.array([[50, 67, 90, 98, 120],
              [55, 71, 93, 102, 129],
              [65, 76, 95, 105, 134],
              [50, 80, 102, 130, 138],
              [60, 82, 97, 135, 151],
              [65, 89, 106, 137, 153],
              [75, 95, 117, 133, 155]], dtype='float64' )

In [14]:
'https://www.centerspace.net/principal-components-regression'

'https://www.centerspace.net/principal-components-regression'

In [15]:
X = np.array([[rd.random() for i in range(5)] for j in range(4)])
Y = np.array([[rd.random()*2] for i in range(4)])
NIPALS_complet(X,Y,3)

array([[-0.07194093],
       [-0.07821926],
       [ 0.05564067],
       [-0.06195667],
       [ 0.00906839]])

In [16]:
def convcheck(a,b):
    return max(np.abs(a-b))
def soft_tresh(Z,kap):
    p,m = np.shape(Z)
    z_soft = np.zeros((p,1))
    if kap < 1 :
        val = np.abs(Z) - kap * max(np.abs(Z))
        z_soft[val>=0]= val[ val>=0 ] * (np.sign(Z))[ val>=0 ]
    return z_soft

def fonc(lam,M,c, kap):
    A = np.linalg.inv(M+lam*np.identity(p)).dot(M).dot(c)
    return A.T.dot(A)-kap**2

def SPLS(X,y,K,kappa,eta,eps=1e-3):
    #https://github.com/cran/spls/blob/master/R/spls.R
    n,p = np.shape(X)
    _,m = np.shape(Y)
    beta = np.zeros((p,m))
    A = set()
    Y1 = Y.copy()
    k = 1
    while k <= K : 
        M = np.dot(np.dot(X.T,Y1),np.dot(Y1.T,X))
        c = np.zeros((p,1))
        diff = 1
        if kappa < 0.5 :
            while diff > eps :  
                kaprime = (1-kappa)/(1-2*kappa)
                lam_etoile = root(fonc,0,args = (M,c,kap))
                w = kaprime*np.linalg.inv((M+lam_etoile*np.identity(p)))
                Z = X.T.dot(Y)
                c0 = c
                c = soft_tresh(M.dot(w),eta)
                diff = convcheck(c,c0)
        else :
            while diff > eps :
                U,_,V = svd(M.dot(c),full_matrices=False)
                w = np.dot(U,V)
                c0 = c
                c = soft_tresh(M.dot(w),eta)
                diff = convcheck(c,c0)
        A = set([i if w[i]!=0 else None for i in range(len(w))]).union([i if beta[i]!=0 else None for i in range(len(beta))])
        if None in A :
            A.remove(None)
        A=list(A)
        beta = NIPALS_complet(X[:,A],y,min(K,len(A))) 
        k+=1
        Y1 = Y-X.dot(beta)
        
    return Y1
            

In [17]:

fichier = open('prostate_ESL.txt','r')
X=[]
for ligne in fichier :
    X.append(ligne.split())

X=np.array(X[1:]).T[1:-1].T
n,p=np.shape(X)

C=np.zeros(np.shape(X))
for i in range(n):
    for j in range(p):
        C[i][j]=float(X[i][j])
X = C[:,:-1]
Y=C[:,-1].reshape(-1,1)


In [20]:
NIPALS_complet(X,Y,8)


array([[ 0.6617092 ],
       [ 0.26510309],
       [-0.15737767],
       [ 0.13958604],
       [ 0.31369926],
       [-0.14751935],
       [ 0.03536545],
       [ 0.1250701 ]])

In [22]:
SPLS(X,Y,8,0.5,0.5)

array([[ 6.86438923],
       [ 8.52005828],
       [ 8.34803204],
       [ 8.66859789],
       [ 8.49863708],
       [ 8.24986604],
       [ 8.92644099],
       [ 7.95410052],
       [ 7.79638048],
       [ 9.731127  ],
       [10.14933851],
       [10.52554441],
       [ 5.41840346],
       [ 9.23625413],
       [ 7.91580227],
       [ 9.77936132],
       [ 7.56793712],
       [ 9.43418925],
       [ 7.29312948],
       [10.83289867],
       [ 9.0353578 ],
       [ 6.05418428],
       [10.14115564],
       [ 1.8955311 ],
       [10.72359405],
       [10.22376366],
       [ 1.77742289],
       [ 8.39385616],
       [ 0.59778735],
       [ 9.85493143],
       [10.2767929 ],
       [10.45389363],
       [10.9393514 ],
       [ 9.42389874],
       [10.97985265],
       [ 8.81942013],
       [ 9.90150901],
       [ 9.19693988],
       [ 5.21155151],
       [ 8.73637314],
       [ 0.16174436],
       [ 9.56940769],
       [10.15958921],
       [ 9.0847623 ],
       [ 7.96471394],
       [ 7